### Parse `Husky`

In [1]:
import mujoco
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../../')
from utils.mujoco_parser import MuJoCoParserClass
from utils.util import r2rpy
import math
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

MuJoCo version:[2.3.7]


/home/rilab/.pyenv/versions/3.8.13/envs/llm/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Parse `scene_husky.xml`

In [2]:
xml_path = '../../asset/husky/scene_husky.xml'
env = MuJoCoParserClass(name='Husky',rel_xml_path=xml_path,VERBOSE=True)
print ("[%s] parsed."%(env.name))

dt:[0.0020] HZ:[500]
n_dof (=nv):[10]
n_geom:[10]
geom_names:['floor', None, None, None, None, None, None, None, None, None]
n_body:[6]
body_names:['world', 'base_husky', 'front_left_wheel_link', 'front_right_wheel_link', 'rear_left_wheel_link', 'rear_right_wheel_link']
n_joint:[5]
joint_names:['base_husky_joint', 'front_left_wheel', 'front_right_wheel', 'rear_left_wheel', 'rear_right_wheel']
joint_types:[0 3 3 3 3]
joint_ranges:
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
n_rev_joint:[4]
rev_joint_idxs:[1 2 3 4]
rev_joint_names:['front_left_wheel', 'front_right_wheel', 'rear_left_wheel', 'rear_right_wheel']
rev_joint_mins:[0. 0. 0. 0.]
rev_joint_maxs:[0. 0. 0. 0.]
rev_joint_ranges:[0. 0. 0. 0.]
n_pri_joint:[0]
pri_joint_idxs:[]
pri_joint_names:[]
pri_joint_mins:[]
pri_joint_maxs:[]
pri_joint_ranges:[]
n_ctrl:[4]
ctrl_names:['front_left_actuator', 'front_right_actuator', 'rear_left_actuator', 'rear_right_actuator']
ctrl_joint_idxs:[7, 8, 9, 10]
ctrl_joint_names:['front_left_wheel', '

### Render

#### Constant `Linear, Angular` Velocity

In [12]:
env.init_viewer(viewer_title='Husky',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=107.08,distance=2.5,elevation=-27,lookat=[0.4,0.3,0.2],
                  VIS_TRANSPARENT=False,VIS_CONTACTPOINT=True,
                  contactwidth=0.1,contactheight=0.1,contactrgba=np.array([1,0,0,1]),
                  VIS_JOINT=True,jointlength=0.5,jointwidth=0.1,jointrgba=[0.2,0.6,0.8,0.6])

# Change controller gain
for ctrl_name in env.ctrl_names:
    kv = 1.0
    env.model.actuator(ctrl_name).gainprm[0] = kv
    env.model.actuator(ctrl_name).biasprm[2] = -kv
    
husky_base_pos_list = []
wheel_radius = 0.17775 / 2.0
wheel_base = 0.2854 * 2
xy_target = [3.0, -4.0]
husky_vel = [0.0, 0.0]  # [m/s, rad/s]

env.reset()

while (env.get_sim_time() < 100.0) and env.is_viewer_alive():
    # Linear and angular velocity
    vx = 1.0
    w = np.pi # np.pi  
    wR = (2 * vx + w * wheel_base) / (2 * wheel_radius)
    wL = (2 * vx - w * wheel_base) / (2 * wheel_radius)
    vel_targets = np.array([wR, wL, wR, wL], dtype=np.float32)
    env.step(ctrl=vel_targets, ctrl_idxs=[0,1,2,3])

    # Do render
    if env.loop_every(HZ=30) or (env.tick == 1):
        # Update viewer information
        p_base = env.get_p_body(body_name='base_husky')
        # env.update_viewer(lookat=p_base,CALL_MUJOCO_FUNC=False)
        # Render contact information
        PLOT_CONTACT = True
        if PLOT_CONTACT:
            p_contacts,f_contacts,geom1s,geom2s,body1s,body2s = env.get_contact_info()
            for (p_contact,f_contact,geom1,geom2) in zip(p_contacts,f_contacts,geom1s,geom2s):
                f_norm = np.linalg.norm(f_contact)
                f_uv = f_contact / (f_norm+1e-8)
                f_len = 0.3 # f_norm*0.05
                label = '' #'[%s]-[%s]'%(geom1,geom2)
                # env.plot_arrow(p=p_contact,uv=f_uv,r_stem=0.01,len_arrow=f_len,rgba=[1,0,0,0.4],label='')
                # env.plot_arrow(p=p_contact,uv=-f_uv,r_stem=0.01,len_arrow=f_len,rgba=[1,0,0,0.4],label='')
                # env.plot_sphere(p=p_contact,r=0.0001,label=label)
        env.plot_T(p=np.zeros(3),R=np.eye(3,3),PLOT_AXIS=True,axis_len=1.0,axis_width=0.01)
        env.plot_body_T(body_name='base_husky',PLOT_AXIS=True,axis_len=0.1,axis_width=0.01)
        env.plot_body_T(body_name='front_left_wheel_link',PLOT_AXIS=True,axis_len=0.1,axis_width=0.01)
        env.plot_body_T(body_name='front_right_wheel_link',PLOT_AXIS=True,axis_len=0.1,axis_width=0.01)
        env.plot_body_T(body_name='rear_left_wheel_link',PLOT_AXIS=True,axis_len=0.1,axis_width=0.01)
        env.plot_body_T(body_name='rear_right_wheel_link',PLOT_AXIS=True,axis_len=0.1,axis_width=0.01)
        env.plot_sphere(p=[xy_target[0], xy_target[1], 0.5],r=0.1,rgba=[1,1,1,1],label='')
        env.plot_T(p=p_base+np.array([0,0,0.5]),R=np.eye(3,3),
                   PLOT_AXIS=False,label='[%.2f]sec'%(env.get_sim_time()))
        
        husky_base_pos_list.append(env.get_p_body('base_husky'))
        for p in np.array(husky_base_pos_list)[::5]:
            env.plot_sphere(p=p+np.array([0,0,0.3]), r=0.01) 
        env.render()

    # Plot
    # if env.loop_every(HZ=0.1) or (env.tick == 1):
    #     scene_img = env.grab_image(resize_rate=0.5)
    #     plt.figure(figsize=(5,4)); plt.imshow(scene_img)
    #     plt.title("Tick:[%d] Time:[%.2f]sec"%(env.tick,env.get_sim_time()),fontsize=9)
    #     plt.show()
    
    # # Print
    # if env.loop_every(HZ=10) or (env.tick == 1):
    #     curr_ctrl = env.data.ctrl
    #     qpos_ctrl = env.data.qpos[env.ctrl_qpos_idxs]
    #     qvel_ctrl = env.data.qvel[env.ctrl_qvel_idxs]
    #     # print ("time:[%.2f]sec, pose:[%s], qvel_ctrl:%s, curr_ctrl:%s"%
    #     #        (env.get_sim_time(),env.get_p_body('base_husky'),qvel_ctrl,curr_ctrl))
        
# Close viewer
env.close_viewer()
print ("Time:[%.2f]sec. Done."%(env.get_sim_time()))

Pressed ESC
Quitting.
Time:[37.60]sec. Done.


#### Target position tracking

In [14]:
env.init_viewer(viewer_title='Husky',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=107.08,distance=2.5,elevation=-27,lookat=[0.4,0.3,0.2],
                  VIS_TRANSPARENT=False,VIS_CONTACTPOINT=True,
                  contactwidth=0.1,contactheight=0.1,contactrgba=np.array([1,0,0,1]),
                  VIS_JOINT=True,jointlength=0.5,jointwidth=0.1,jointrgba=[0.2,0.6,0.8,0.6])

husky_base_pos_list = []
wheel_radius = 0.17775 / 2.0
wheel_base = 0.2854 * 2
# Define the velocity gains
v_max = 5.0  # maximum linear velocity, in m/s
w_max = 50.0  # maximum angular velocity, in rad/s

p_target = np.array([5, -1.5, 0.5])


vx_lst, w_lst = [], []
env.reset()
while (env.get_sim_time() < 100.0) and env.is_viewer_alive():

    # Get the current position of the vehicle from the simulation
    p_base = env.get_p_body('base_husky')
    d_pos = p_target[:2] - p_base[:2]
    # Calculate the distance to the target point
    distance = np.linalg.norm(d_pos)
    if distance < 0.1:
        break

    R_base = r2rpy(env.get_R_body('base_husky'))
    heading = np.arctan2(d_pos[1], d_pos[0]) - R_base[2]
    heading = np.mod(heading + np.pi, 2*np.pi) - np.pi
    
    # compute desired linear and angular velocities
    linear_vel = v_max * np.tanh(distance)
    angular_vel = w_max * np.tanh(heading)
    
    # compute wheel speeds for forward motion
    vl = linear_vel - (angular_vel * wheel_base / 2)
    vr = linear_vel + (angular_vel * wheel_base / 2)

    ctrl = [vl,vr,vl,vr]
    # Update the simulation state
    env.step(ctrl, ctrl_idxs=[0,1,2,3])

    # Do render
    if env.loop_every(HZ=30) or (env.tick == 1):
        # Update viewer information

        env.plot_sphere(p=p_target,r=0.1,rgba=[1,1,1,1],label='')
        env.plot_T(p=p_base+np.array([0,0,0.5]),R=np.eye(3,3),
                   PLOT_AXIS=False,label='[%.2f]sec'%(env.get_sim_time()))
        
        husky_base_pos_list.append(env.get_p_body('base_husky'))
        for p in np.array(husky_base_pos_list)[::5]:
            env.plot_sphere(p=p+np.array([0,0,0.3]), r=0.01) 
        env.render()
    
# Close viewer
env.close_viewer()
print ("Time:[%.2f]sec. Done."%(env.get_sim_time()))



Pressed ESC
Quitting.
Time:[22.18]sec. Done.


### Path following: just update the index of reference path

In [17]:
env.init_viewer(viewer_title='Husky',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=107.08,distance=2.5,elevation=-27,lookat=[0.4,0.3,0.2],
                  VIS_TRANSPARENT=False,VIS_CONTACTPOINT=True,
                  contactwidth=0.1,contactheight=0.1,contactrgba=np.array([1,0,0,1]),
                  VIS_JOINT=True,jointlength=0.5,jointwidth=0.1,jointrgba=[0.2,0.6,0.8,0.6])

husky_base_pos_list = []
wheel_radius = 0.17775 / 2.0
wheel_base = 0.2854 * 2
# Define the velocity gains
v_max = 5.0  # maximum linear velocity, in m/s
w_max = 50.0  # maximum angular velocity, in rad/s

# Define the target path
path_x = np.arange(2, 50, 0.5)
path_y = [math.cos(ix / 5.0) * ix / 2.0 for ix in path_x]
path_z = [0.0 for ix in path_x]
path = np.array([path_x, path_y, path_z]).T
p_target = path[0]
path_idx = 0

vx_lst, w_lst = [], []
env.reset()
while (env.get_sim_time() < 100.0) and env.is_viewer_alive():

    # Get the current position of the vehicle from the simulation
    p_base = env.get_p_body('base_husky')
    husky_base_pos_list.append(p_base)
    d_pos = p_target[:2] - p_base[:2]
    # Calculate the distance to the target point
    distance = np.linalg.norm(d_pos)

    if distance < 0.3:
        path_idx += 1
        p_target = path[path_idx]

    R_base = r2rpy(env.get_R_body('base_husky'))
    heading = np.arctan2(d_pos[1], d_pos[0]) - R_base[2]
    heading = np.mod(heading + np.pi, 2*np.pi) - np.pi
    
    # compute desired linear and angular velocities
    linear_vel = v_max * np.tanh(distance)
    angular_vel = w_max * np.tanh(heading)
    
    # compute wheel speeds for forward motion
    vl = linear_vel - (angular_vel * wheel_base / 2)
    vr = linear_vel + (angular_vel * wheel_base / 2)

    ctrl = [vl,vr,vl,vr]
    # Update the simulation state
    env.step(ctrl, ctrl_idxs=[0,1,2,3])

    # Do render
    if env.loop_every(HZ=30) or (env.tick == 1):
        # Update viewer information
        env.plot_T(p=p_base+np.array([0,0,0.5]),R=np.eye(3,3),
                   PLOT_AXIS=False,label='[%.2f]sec'%(env.get_sim_time()))
        p_base_marker = np.array([p_base[0],p_base[1],0.4])
        p_target_marker = np.array([p_target[0],p_target[1],0.4])
        for p in path: env.plot_sphere(p=np.array([p[0],p[1],0.4]),r=0.05,rgba=[0.05,0.05,0.05,1])
        for p in np.array(husky_base_pos_list)[::70]:
            env.plot_sphere(p=p+np.array([0,0,0.3]), r=0.01) 
        env.plot_sphere(p=p_base_marker,r=0.05,rgba=[0.05,0.95,0.05,1], label='Current Base position')
        env.plot_sphere(p=p_target_marker,r=0.05,rgba=[0.95,0.05,0.05,1], label='Current Target position')
        env.render()
    
# Close viewer
env.close_viewer()
print ("Time:[%.2f]sec. Done."%(env.get_sim_time()))

Pressed ESC
Quitting.
Time:[33.73]sec. Done.
